In [74]:
import os
os.getcwd()

'/data/zhaozibo/text-kgs-table/components/ft_data_prep'

In [75]:
import sys, os
sys.path.append('/data/zhaozibo/text-kgs-table')
import pandas as pd
from components.kgs.dataset_kgs.e2e_kg import *
from utils import visualize_knowledge_graph_interactive, KnowledgeGraph, visualize_knowledge_graph
from components.retriever.DocData import *

In [111]:
part = "test"
file_path = f'../../data/rotowire/{part}.text'

# Reading the file line by line and storing each line in a list
lines = []
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        lines.append(line.strip())
lines[0]

"The Atlanta Hawks (46 - 12) beat the Orlando Magic (19 - 41) 95 - 88 on Friday. Al Horford had a good all - around game, putting up 17 points, 13 rebounds, four assists and two steals in a tough matchup against Nikola Vucevic. Kyle Korver was the lone Atlanta starter not to reach double figures in points. Jeff Teague bounced back from an illness, he scored 17 points to go along with seven assists and two steals. After a rough start to the month, the Hawks have won three straight and sit atop the Eastern Conference with a nine game lead on the second place Toronto Raptors. The Magic lost in devastating fashion to the Miami Heat in overtime Wednesday. They blew a seven point lead with 43 seconds remaining and they might have carried that with them into Friday's contest against the Hawks. Vucevic led the Magic with 21 points and 15 rebounds. Aaron Gordon (ankle) and Evan Fournier (hip) were unable to play due to injury. The Magic have four teams between them and the eighth and final play

In [67]:
from components.kgs.dataset_kgs.e2e_rotowire_field import fields_list_rotowire, fields_list_e2e
from components.kgs.dataset_kgs.CPL_field import fields_list_CPL
import json
import numpy as np
import pandas as pd

In [112]:
fields_list_rotowire['Table1']

{'Row_Name': {'Team ': 2},
 'Fields': {'Team name': str,
  'Number of team assists': int,
  'Percentage of 3 points': int,
  'Percentage of field goals': int,
  'Losses': int,
  'Total points': int,
  'Points in 1st quarter': int,
  'Points in 2nd quarter': int,
  'Points in 3rd quarter': int,
  'Points in 4th quarter': int,
  'Rebounds': int,
  'Turnovers': int,
  'Wins': int}}

In [131]:
ie_prompt_extract_role_template="""You are a useful assistant who can classify given proper nouns based on the list of role options and related information provided by the user.
1. Determine whether this proper noun belongs to any of the given role lists. If it does not belong to any of them, respond 'Not Role'.
2. If it belongs to a certain role in the role list, you only need to respond the options in the list. You cannot output any content not in the role list.
3. Respond to the user's question like the examples provided below:

Proper noun: Hawks
Related Context: The Atlanta Hawks (46 - 12) beat the Orlando Magic (19 - 41) 95 - 88 on Friday. Al Horford had a good all - around game, putting up 17 points, 13 rebounds.
Role option scope: ['Team']
Question: Dose the proper noun is a Role in the scope? If dose, which is?
Answer: Team

Proper noun: Vaults
Related Context: The Vaults pub near Café Adriatic has a 5 star rating.  Prices start at £30.
Role option scope: ['Restaurant']
Question: Dose the proper noun is a Role in the scope? If dose, which is?
Answer: Restaurant

Proper noun: Blue
Related Context: Close to Café Brazil, The Cambridge Blue pub serves delicious Tuscan Beef for the cheap price of £10.50. Delicious Pub food.
Role option scope: ['Team', 'Player']
Question: Dose the proper noun is a Role in the scope? If dose, which is?
Answer: Not Role

Below is the usr's question: 

Proper noun: {PRNON}
Related Context: {RELATED_CONTEXT}
Role option scope: {SCOP}
Answer:
"""

In [130]:
ie_prompt_extract_value_template="""You are a useful table content filling assistant that can extract the attribute field values of the role based on the row and column values provided by the user, as well as their corresponding paragraph original text, from the original text.
1. Check if the provided paragraph contains the attribute values corresponding to the role. If not, respond 'Bad Infomation'.
2. If the relevant paragraph contains attribute values for the role, respond the value according to the given requirements.
3. Respond to the user's question like the examples provided below:

Role: Taste of Cambridge
Attribute: Price range
Related Context: Taste of Cambridge is a restaurant with a customer rating of 3 out of 5 and and a price range of £20-£25
Value scope: ['More than £30', 'Cheap', 'Less than £20', '£20-25', 'Moderate', 'High']
Question: What's the value of Taste of Cambridge's Price range?
Answer: £20-£25

Role: James Harden
Attribute: Points
Related Context: It was another day at work on Tuesday for superstar James Harden, who led the Rockets to a blowout victory with 34 points and 11 assists.
Value scope: integer
Question: What's the value of Player's Points?
Answer: 34

Role: Suns
Attribute: Number of team assists
Related Context: Memphis also registered 25 assists compared to only 13 - on 32 field goals - for the Suns
Value scope: integer
Question: What's the value of Suns's Number of team assists?
Answer: 13

Below is the usr's question:
    Role: {ROLE}
    Attribute: {FIELD}
    Related Context: {RELATED_CONTEXT}
    Value scope: {SCOPE}
    Question: What's the value of {ROLE}'s {FIELD}?
    Answer:
"""

In [129]:
ie_prompt_extract_role_name_rotowire_template="""You are a useful table content filling assistant that can extract the attribute field values of the role based on the row and column values provided by the user, as well as their corresponding paragraph original text, from the original text.
1. Check if the provided paragraph contains the attribute values corresponding to the role. If not, respond 'Bad Infomation'.
2. If the relevant paragraph contains attribute values for the role, respond the value according to the given requirements.
3. Respond to the user's question like the examples provided below:

Role: Home team
Attribute: Team Name
Related Context: The Los Angeles Clippers (3 - 1) defeated the Utah Jazz (1 - 3) 107 - 101 on Monday. Chris Paul recorded the NBA's first triple - double of the 2014 - 15 season, putting up 13 points, 10 rebounds, and 12 assists in 36 minutes. He also threw in two steals and a blocked shot for good measure. Teammate Blake Griffin helped Paul convert some of those assists, shooting 14 - of - 21 from the field and finishing with 31 points in 38 minutes. J.J. Redick started at shooting guard but played a season - low 16 minutes as coach Doc Rivers opted to go with Jamal Crawford most of the night. Crawford missed the Clippers' last game with a rib injury but returned to put up 19 points off the bench in 34 minutes. Gordon Hayward put up a LeBron James-esque line of 27 points, seven rebounds, and five assists in 36 minutes. He was aided in keeping the game close by starting power forward Enes Kanter who put up 17 points and nine rebounds in 27 minutes. The Jazz were relatively efficient with their shooting, but they committed 17 turnovers in contrast to the Clippers' 10, giving away the game in some respects. The Clippers' next game is against the Golden State Warriors on Wednesday, while the Jazz will return home to face the Cleveland Cavaliers on Wednesday.
Question: What's is the Team Name of Home team
Answer: Los Angeles Clippers

Role: Visiting team
Attribute: Team Name
Related Context: The Los Angeles Clippers (3 - 1) defeated the Utah Jazz (1 - 3) 107 - 101 on Monday. Chris Paul recorded the NBA's first triple - double of the 2014 - 15 season, putting up 13 points, 10 rebounds, and 12 assists in 36 minutes. He also threw in two steals and a blocked shot for good measure. Teammate Blake Griffin helped Paul convert some of those assists, shooting 14 - of - 21 from the field and finishing with 31 points in 38 minutes. J.J. Redick started at shooting guard but played a season - low 16 minutes as coach Doc Rivers opted to go with Jamal Crawford most of the night. Crawford missed the Clippers' last game with a rib injury but returned to put up 19 points off the bench in 34 minutes. Gordon Hayward put up a LeBron James-esque line of 27 points, seven rebounds, and five assists in 36 minutes. He was aided in keeping the game close by starting power forward Enes Kanter who put up 17 points and nine rebounds in 27 minutes. The Jazz were relatively efficient with their shooting, but they committed 17 turnovers in contrast to the Clippers' 10, giving away the game in some respects. The Clippers' next game is against the Golden State Warriors on Wednesday, while the Jazz will return home to face the Cleveland Cavaliers on Wednesday.
Question: What's is the Team Name of Visiting team
Answer: Utah Jazz

Below is the usr's question:
    Role: {ROLE}
    Attribute: {FIELD}
    Related Context: {RELATED_CONTEXT}
    Question: What's the {FIELD} of {ROLE}?
    Answer:

"""

In [120]:
fields = fields_list_rotowire['Table1']['Fields']

In [132]:
ft_data = []

In [133]:
corresponding_table = pd.read_excel(os.path.join("/data/zhaozibo/", f"text-kgs-table/data/rotowire/tables/test/0.xlsx"))

In [134]:
import warnings
# Ignore all warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import random

for i in tqdm(range(len(lines))):
    team_role = "Home team"
    field = "Team name"
    related_context = lines[i]
    corresponding_table = pd.read_excel(os.path.join("/data/zhaozibo/", f"text-kgs-table/data/rotowire/tables/{part}/{i}.xlsx"))
    corresponding_table = corresponding_table.rename(columns={"Unnamed: 0":"Team Label"})
    corresponding_table = corresponding_table.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    corresponding_table = corresponding_table.set_index("Team Label")
    filled_template_role_hometeam_prompt = ie_prompt_extract_role_name_rotowire_template.format(ROLE=team_role, FIELD=field, RELATED_CONTEXT=related_context)
    filled_template_role_hometeam_prompt_answer = corresponding_table.loc[team_role, "Team name"]
    ft_data.append({"instruction":"Given the context, extract the team name", "input":filled_template_role_hometeam_prompt, "output":str(filled_template_role_hometeam_prompt_answer)})


    for attribute, scope in fields.items():
        if attribute == "Team name":
            continue
        role = corresponding_table.loc[team_role, "Team name"]
        role_value_prompt = ie_prompt_extract_value_template.format(ROLE=role, FIELD=attribute, SCOPE=scope, RELATED_CONTEXT=lines[i])
        role_value_answer = corresponding_table.loc[team_role, attribute]
        if pd.isna(role_value_answer):
            ft_data.append({"instruction":"Given the context, extract the information", "input":role_value_prompt, "output":"Bad Infomation"})
        else:
            ft_data.append({"instruction":"Given the context, extract the information", "input":role_value_prompt, "output":str(role_value_answer)})

    team_role = "visiting team"
    field = "Team name"
    related_context = lines[i]
    filled_template_role_hometeam_prompt = ie_prompt_extract_role_name_rotowire_template.format(ROLE=team_role, FIELD=field, RELATED_CONTEXT=related_context)
    filled_template_role_hometeam_prompt_answer = corresponding_table.loc[team_role, "Team name"]
    ft_data.append({"instruction":"Given the context, extract the team name", "input":filled_template_role_hometeam_prompt, "output":str(filled_template_role_hometeam_prompt_answer)})

    for attribute, scope in fields.items():
        if attribute == "Team name":
            continue
        role = corresponding_table.loc[team_role, "Team name"]
        role_value_prompt = ie_prompt_extract_value_template.format(ROLE=role, FIELD=attribute, SCOPE=scope, RELATED_CONTEXT=lines[i])
        role_value_answer = corresponding_table.loc[team_role, attribute]
        if pd.isna(role_value_answer):
            if random.randint(1, 4) == 1:
                ft_data.append({"instruction":"Given the context, extract the information", "input":role_value_prompt, "output":"Bad Infomation"})
        else:
            ft_data.append({"instruction":"Given the context, extract the information", "input":role_value_prompt, "output":str(role_value_answer)})

  0%|          | 0/728 [00:00<?, ?it/s]

100%|██████████| 728/728 [00:06<00:00, 111.37it/s]


In [119]:
corresponding_table

,Team name,Number of team assists,Percentage of 3 points,Percentage of field goals,Losses,Total points,Points in 1st quarter,Points in 2nd quarter,Points in 3rd quarter,Points in 4th quarter,Rebounds,Turnovers,Wins
Team Label,,,,,,,,,,,,,
Home team,Hawks,NaN,NaN,NaN,12,95,NaN,NaN,NaN,0,NaN,NaN,46
visiting team,Magic,NaN,NaN,NaN,41,88,NaN,NaN,NaN,21,NaN,NaN,19


In [ ]:
import warnings
from tqdm import tqdm
import random
# Ignore all warnings
warnings.filterwarnings('ignore')

for i in tqdm(range(len(lines)), desc='obv'):
    related_context = lines[i]
    corresponding_table = pd.read_excel(os.path.join("/data/zhaozibo/", f"text-kgs-table/data/rotowire/tables/{part}/{i}.xlsx"), sheet_name='team')
    corresponding_table = corresponding_table.rename(columns={"Unnamed: 0":"Team Label"})
    corresponding_table = corresponding_table.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    
    for j in range(corresponding_table.shape[0]):
        player_name = corresponding_table.loc[j, "Name"]
        if pd.isna(player_name):
            continue
        for attribute, scope in fields.items():
            if attribute == "Name":
                continue
            role = player_name
            role_value_prompt = ie_prompt_extract_value_template.format(ROLE=role, FIELD=attribute, SCOPE=scope, RELATED_CONTEXT=lines[i])
            role_value_answer = corresponding_table.loc[j, attribute]
            if pd.isna(role_value_answer):
                if random.randint(1, 4) == 1:
                    ft_data.append({"instruction":"Given the context, extract the information", "input":role_value_prompt, "output":"Bad Infomation"})
            else:
                ft_data.append({"instruction":"Given the context, extract the information", "input":role_value_prompt, "output":str(role_value_answer)})


In [135]:
ft_data[:20]

[{'instruction': 'Given the context, extract the team name',
  'input': "You are a useful table content filling assistant that can extract the attribute field values of the role based on the row and column values provided by the user, as well as their corresponding paragraph original text, from the original text.\n1. Check if the provided paragraph contains the attribute values corresponding to the role. If not, respond 'Bad Infomation'.\n2. If the relevant paragraph contains attribute values for the role, respond the value according to the given requirements.\n3. Respond to the user's question like the examples provided below:\n\nRole: Home team\nAttribute: Team Name\nRelated Context: The Los Angeles Clippers (3 - 1) defeated the Utah Jazz (1 - 3) 107 - 101 on Monday. Chris Paul recorded the NBA's first triple - double of the 2014 - 15 season, putting up 13 points, 10 rebounds, and 12 assists in 36 minutes. He also threw in two steals and a blocked shot for good measure. Teammate Blak

In [137]:
# Specify the filename
ft_data_path = "/data/zhaozibo/LLaMA-Factory/data"
filename = 'rotowire_ie_train.json'
file_path = os.path.join(ft_data_path, filename)
# Open the file in write mode
with open(file_path, 'w') as json_file:
    json.dump(ft_data, json_file)

In [138]:
len(ft_data)

14463

In [128]:
ft_data

[{'instruction': 'Given the context, extract the team name',
  'input': "\nYou are a useful table content filling assistant that can extract the attribute field values of the role based on the row and column values provided by the user, as well as their corresponding paragraph original text, from the original text.\n1. Check if the provided paragraph contains the attribute values corresponding to the role. If not, respond 'Bad Infomation'.\n2. If the relevant paragraph contains attribute values for the role, respond the value according to the given requirements.\n3. Respond to the user's question like the examples provided below:\n\nRole: Home team\nAttribute: Team Name\nRelated Context: The Los Angeles Clippers (3 - 1) defeated the Utah Jazz (1 - 3) 107 - 101 on Monday. Chris Paul recorded the NBA's first triple - double of the 2014 - 15 season, putting up 13 points, 10 rebounds, and 12 assists in 36 minutes. He also threw in two steals and a blocked shot for good measure. Teammate Bl

In [43]:
dict(corresponding_table.loc[0, :])
ie_prompt_extract_role_name_rotowire_template.format()

{'Team Label': 'Home team ',
 'Team name': 'Clippers',
 'Number of team assists': nan,
 'Percentage of 3 points': nan,
 'Percentage of field goals': nan,
 'Losses': 1,
 'Total points': 107,
 'Points in 1st quarter': nan,
 'Points in 2nd quarter': nan,
 'Points in 3rd quarter': nan,
 'Points in 4th quarter': nan,
 'Rebounds': nan,
 'Turnovers': 10,
 'Wins': 3}

In [36]:
len(os.listdir(os.path.join("/data/zhaozibo/", "text-kgs-table/data/rotowire/tables/train"))), len(lines)

(3398, 3398)